## socket模块

网络上的两个程序通过一个双向的通信连接进行数据的交换，连接的一端就是一个socket，也叫套接字，基本就是一个信息通道

- **套接字：**服务端套接字、客户端套接字
- **实例化套接字：**（普通套接字不用提供任何参数）
    - 地址族（默认socket.AF_INET）
    - 流套接字（默认socket.SOCK_STREAM），还是数据报套接字（socket.SOCK_DGRAM）
    - 协议（默认值0）
- **服务器套接字的使用**
    - 先调用bind方法，将主机名和端口绑定
    - 再调用listen方法监听特定地址， 接受一个参数，设置在队列中等待的连接数，超过就拒绝连接
    - 调用accept方法等待接收客户端连接，客户端连接到来时停止等待，返回格式为(client, address)的元组，client为客户端套接字，address为（host, port）
- **客户端套接字的使用**
    - 调用connect方法，并提供指定地址（在服务端使用socket.gethostname函数获取主机名），格式为(host, port)的元组
- **区别**
    - 服务器套接字必须随时准备连接的到来，还必须处理多个连接
    - 客户端套接字只需连接，任务完成后断开连接即可
- **传输数据**
    - send方法，提供一个字符串
    - recv方法，设置接收数据的最大值，单位为字节，1024
- **端口限制**
    - Linux或UNIX系统中，需要管理员权限才能使用1024以下的端口，这些端口供标准服务使用
    - 80端口供Web服务器使用

### 获取内网和公网的ip

In [29]:
import requests
import socket
import re
r = requests.get('http://www.net.cn/static/customercare/yourip.asp')
s = re.search('<h2>(.+)</h2>', r.text)
print('公网IP：', s.group(1))
print('局域网IP：', socket.gethostbyname(socket.gethostname()))

公网ip： 58.198.250.196
局域网ip： 169.254.22.239


### 创建最简单的服务器和客户端

**注意：**下面两个文件要先执行server.py，再执行client.py  
**server.py**
***
<code>
import socket

s = socket.socket()

host = socket.gethostname()
port = 1234

s.bind((host, port))
s.listen(1)

client, address = s.accept()
while True:
    get_words = client.recv(1024)
    print('装疯卖傻耍赖皮:\n', bytes.decode(get_words))
    ret_words = input('我：\n')
    client.send(bytes(ret_words, encoding='utf8'))
</code>

**client.py**
***
<code>
import socket

s = socket.socket()  # 实例化客户端套接字

host = socket.gethostname()  
port = 1234

s.connect((host, port))  # 发起连接请求
while True:
    words = input('我：\n')
    s.send(bytes(words, encoding='utf8'))
    print('小帅哥：\n', bytes.decode(s.recv(1024)))  # 输出接收到的数据
</code>

### SocketServer及相关的类

- SocketServer是标准库提供的服务器框架的基石
- SocketServer包含四个基本的服务器
    - TCPServer （支持TCP套接字流）
    - UDPServer （支持UDP数据报套接字）
    - UnixStreamServer
    - UnixDatagramServer
- 大部分代码运行在请求处理器中
    - 基本请求处理程序类BaseRequestHandler，将所有操作都放在一个方法中——服务器调用的方法handle，通过属性self.request来访问客户端套接字
    - StreamRequestHandler类，它包含另外两个属性：self.rfile（用于读取）和self.wfile（用于写入），可使用这两个类似于文件的对象来与客户端通信

In [ ]:
from socketserver import TCPServer, StreamRequestHandler
class Handler(StreamRequestHandler):
    def handle(self):
        addr = self.request.getpeername()
        print('Got connection from ', addr)
        self.wfile.write('Thank you for connecting.')


server = TCPServer((socketserver.socket.gethostname(), 1234), Handler)
server.serve_forever()

## 多个连接

处理多个连接
1. 分叉
2. 线程化
3. 异步I/O

### 分叉

**原理**
1. 对进程进行分叉，就是对进程进行**复制**，且每个进程都有自己的**内存副本**
2. 分叉后的进程从**当前位置**继续运行
3. 原进程称为**父进程**，复制的进程称为**子进程**，可通过fork函数判断
***
**实例**
1. 对于每个客户端连接，通过分叉创建一个子进程
2. 父进程继续监听连接，子进程负责处理客户端请求
3. 子进程处理完客户端请求，即可退出
4. 子进程间都是并行运行，客户端无需等待，占用资源多
***
**windows不支持分叉**

#### 分叉处理器

In [ ]:
from socketserver import TCPServer, ForkingMixIn, StreamRequestHandler
class Server(ForkingMixIn, TCPServer): pass
class Handler(StreamRequestHandler):
def handle(self):
addr = self.request.getpeername()
print('Got connection from', addr)
self.wfile.write('Thank you for connecting')
server = Server(('', 1234), Handler)
server.serve_forever()

### 线程化

- 线程是轻量级的进程（子进程），同时位于一个进程中共享内存
- **线程共享内存**：可能会干扰内存空间，处理复杂

#### 线程化处理器

In [ ]:
from socketserver import TCPServer, ThreadingMixIn, StreamRequestHandler
class Server(ThreadingMixIn, TCPServer): pass
class Handler(StreamRequestHandler):
def handle(self):
addr = self.request.getpeername()
print('Got connection from', addr)
self.wfile.write('Thank you for connecting')
server = Server(('', 1234), Handler)
server.serve_forever()

### 异步I/O

- 只处理当前正在通信的客户端，甚至无需不断监听，只需监听后将客户端加入队列即可
- **select**模块
    - select函数包含三个必选参数和一个可选参数
        - 必选参数：三个序列，等待的输入连接、等待的输出连接、等待的错误连接
        - 可选参数：超时时间
    - poll函数（windows不支持）

## twisted框架

异步网络编程框架，可以实现事件处理程序

In [ ]:
from twisted.internet import reactor
from twisted.internet.protocol import Factory
from twisted.protocols.basic import LineReceiver

class SimpleLogger(LineReceiver):
    def connectionMade(self):
        print('Got connection from', self.transport.client)
    def connectionLost(self, reason):
        print(self.transport.client, 'disconnected')
    def lineReceived(self, line):
        print(line)

factory = Factory()
factory.protocol = SimpleLogger
reactor.listenTCP(1234, factory)
reactor.run()